# Module 25: Group-level Analysis 3

### Statistical Analysis

There are 3 main parts to a statistical analysis:
<ol>
<li>Model - describes the parameters of the unknown distribution thought to be generating the data
<li>Method - defines the loss function that is minimized in order to find the unknown model parameters
<li>Algorithm - defines the manner in which the chosen loss function is minimized
</ol>

Before performing group analysis, motion correction (intrasubject registration), spatial normalization (intersubject registration), and spatial smoothing (overcome limitations in spatial normalization)
should be done.

In group analysis we need the voxels to align across subjects.

### Model Creation

#### Level 1

In level 1 we use the standard GLM analysis:
<img src='level1.png'>

In this level we have autocorrelated data, but we have a relatively large number of observations.

We can combine all of these subjects' data by concatenating the Y, X, beta, e, and V (covariance) matrices as follows:
<img src='matrices.png'>

Therefore, we could fit each subjects data individually, but for simplicity in our records, we can write the full first-level model as:
<img src='fulllevel1.png'>

#### Level 2

We can now write the full second level model as:
<img src='fulllevel2.png'>

where $X_g$ is the second level design matrix (separating cases and controls) and $\beta_g$ is the vector of second level parameters. In the second level we usually have independent and identically distributed data, but relatively few observations.

Note that $X_g\beta_g$ includes whether or not the subject is a case or control AND the amplitude for cases and controls. In matrix algebra this looks like this:

In [2]:
import numpy as np
#create design matrix
X_g = np.array([[1,0],[1,0],[0,1],[0,1]])
#create parameter vector (case/control amplitudes)
B_g = np.array([3,7])
print(X_g, B_g)

[[1 0]
 [1 0]
 [0 1]
 [0 1]] [3 7]


In [3]:
#multiply the matrices to get the final beta vector
XB_g = X_g * B_g
print(XB_g)

[[3 0]
 [3 0]
 [0 7]
 [0 7]]


As you can see, we can do this to relate subject specific parameters contained in in $\beta$ to population parameters $\beta_g$. We assume that the first level parameters are randomly sampled from a population of possible regression parameters, which allows us to generalize the results to the whole population.
<img src='sourcepop.png'>

Given our summary equations for the two levels of models above, we can combine the two levels into a single level:
<img src='mixedmodel.png'>

saying that Y follows a distribution with the above mean and variance.

### Method

To find the parameters of the above model, we need to define a loss function to minimize. 

In many cases Maximum Likelihood Estimation (MLE) or the Restricted Maximum Likelihood Estimation (RMLE) are used.

#### Maximum Likelihood Estimation

<ul>
<li>Maximizes the likelihood of the <b>data</b>
<li>Produces <b>biased</b> estimates of the variance components
</ul>

#### Restricted Maximum Likelihood Estimation

<ul>
<li>Maximizes the likelihood of the <b>residuals</b>
<li>Produces <b>unbiased</b> estimates of the variance components
</ul>

### Algorithms

To minimize the lost function from above, we use an algorithm. In many cases these will be Newton-Raphson, Fisher-scoring, EM-algorithm, or IGLS/RIGLS.

#### Newton-Raphson

Iterative procedure that finds estimates using the derivatives at the current solution

#### Fisher Scoring

Very similar to Newton-Raphson, but finds the estimates using Fisher Information.

#### EM-algorithm

Iterative procedure that finds estimates from models that depend on unobserved latent variables such as the second level error.

Ultimately, which methods and algorithms you use will depend on which software packages you decide to utilize. Different packages implement different algorithms like the ones discussed above. 

### Software

Different neuroimaging software packages have implemented different mixed-effects models. They differ in which methods and algorithms to use. 

But the most commonly used approach in fMRI analysis is a simple non-iterative two-stage least squares approach (the summary statistics approach). Recall that results from the first level are reused in the second level, reducing the computational burden of fitting an entire model.

### Summary Statistics Approach

<ol>
<li>Fit a model to each subject's data
<li>Construct contrast images for each subject
<li>Conduct a t-test on the contrast images
</ol>

However, only one contrast can be estimated at a time and only the contrasts, and NOT the variance components are recycled from level 1.

<b>Assumptions</b>:
<ul>
<li>Homogeneous intra-subject variance
<li>Balanced designs
</ul>

### Group Analysis

Using temporal basis sets at the first level means it can be difficult to summarize the response with a single number, making group inference difficult.

Instead, we can perform a group analysis using 
<ul>
<li>the "main" basis function
<li>all basis functions (and do an F-test)
<li>re-parameterized fitted responses - recreate the HRF and estimate the magnitude, and use this information at the second level.
</ul>